In [5]:
import sys
import pandas as pd
import numpy as np
from scipy.stats import weibull_min
from scipy.optimize import minimize
import warnings

C:\Users\jhroe\AppData\Local\Temp\ipykernel_13928\1706878130.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [57]:
input_file = "./results/step1_1nn_output.csv"
output_file = "./results/step2_weibull_initial_params.csv"

In [14]:
def fit_weibull(data):
    # Function to fit Weibull distribution and return shape and scale parameters
    def negative_log_likelihood(params, data):
        shape, scale = params
        return -np.sum(weibull_min.logpdf(data, shape, scale=scale))

    initial_params = [1, 1]
    result = minimize(negative_log_likelihood, initial_params, args=(data,), method='Nelder-Mead')
    if result.success:
        return result.x
    else:
        return [np.nan, np.nan]

In [58]:
all_distances_data = pd.read_csv(input_file, sep=',').dropna()
all_distances_data

,Patient_ID,phenotype_from,phenotype_to,count,bin,WinMean,count_scaled,phenotype_combo
0,LUAD_D001,B cell,B cell,0,0,2.0,0.000000,B cell_to_B cell
1,LUAD_D001,B cell,B cell,0,1,3.0,0.000000,B cell_to_B cell
2,LUAD_D001,B cell,B cell,0,2,4.0,0.000000,B cell_to_B cell
3,LUAD_D001,B cell,B cell,2,3,5.0,0.015385,B cell_to_B cell
4,LUAD_D001,B cell,B cell,8,4,6.0,0.061538,B cell_to_B cell
...,...,...,...,...,...,...,...,...
4852678,LUAD_D416,Treg,Treg,1,292,294.0,0.004310,Treg_to_Treg
4852679,LUAD_D416,Treg,Treg,1,293,295.0,0.004310,Treg_to_Treg
4852680,LUAD_D416,Treg,Treg,0,294,296.0,0.000000,Treg_to_Treg
4852681,LUAD_D416,Treg,Treg,0,295,297.0,0.000000,Treg_to_Treg


In [16]:
# Load 1-NN X/Y histogram coordinates dataframe (output from script 1__get1NNdistances.R)

# When a cell doest occur in an image it generates nans which are dropped here
all_distances_data['distance_window'] = all_distances_data['WinMean']
all_distances_data['phenotype_combo'] = all_distances_data['phenotype_from'] + '_to_' + all_distances_data['phenotype_to']
all_distances_data = all_distances_data[['Patient_ID', 'phenotype_combo', 'count_scaled', 'distance_window']]
all_distances_data['new'] = all_distances_data['count_scaled'] * 1000
# print(all_distances_data[all_distances_data.isna().any(axis=1)])
# print("Max and nans in 'new'",all_distances_data['new'].max(), all_distances_data['new'].isna().sum())
all_distances_data['new'] = all_distances_data['new'].round().astype(int)
all_distances_data

,Patient_ID,phenotype_combo,count_scaled,distance_window,new
0,LUAD_D001,B cell_to_B cell,0.000000,2.0,0
1,LUAD_D001,B cell_to_B cell,0.000000,3.0,0
2,LUAD_D001,B cell_to_B cell,0.000000,4.0,0
3,LUAD_D001,B cell_to_B cell,0.015385,5.0,15
4,LUAD_D001,B cell_to_B cell,0.061538,6.0,62
...,...,...,...,...,...
4852678,LUAD_D416,Treg_to_Treg,0.004310,294.0,4
4852679,LUAD_D416,Treg_to_Treg,0.004310,295.0,4
4852680,LUAD_D416,Treg_to_Treg,0.000000,296.0,0
4852681,LUAD_D416,Treg_to_Treg,0.000000,297.0,0


In [21]:
output_file

'./results/step2_weibull_initial_params.csv'

In [22]:
# After 60 hours the script got to LUAD_D098 
# Save the intermediate result
# initial_params

all_combos_dists.to_csv('./results/step2_intermediate_result.csv', sep=',', index=False)

In [55]:
all_combos_dists

,dists,Patient_ID,combo
0,1.0,LUAD_D001,B cell_to_B cell
1,5.0,LUAD_D001,B cell_to_B cell
2,5.0,LUAD_D001,B cell_to_B cell
3,5.0,LUAD_D001,B cell_to_B cell
4,5.0,LUAD_D001,B cell_to_B cell
...,...,...,...
16354507,294.0,LUAD_D416,Treg_to_Treg
16354508,295.0,LUAD_D416,Treg_to_Treg
16354509,295.0,LUAD_D416,Treg_to_Treg
16354510,295.0,LUAD_D416,Treg_to_Treg


In [33]:
# New optimzied

# Initialize an empty list to collect DataFrames
all_combos_dists_list = []

# Group the data by 'Patient_ID' and 'phenotype_combo'
grouped = all_distances_data.groupby(['Patient_ID', 'phenotype_combo'])

# Iterate over each group
for (tn, combo), df_filtered in grouped:
    dists = [1] + df_filtered['distance_window'].tolist()
    times = [1] + df_filtered['new'].tolist()

    if len(times) == 2:
        dists = list(range(1, 299))
        times = [0] * 298

    # Use itertools to efficiently repeat elements
    from itertools import chain, repeat
    all_dists_tnum = list(chain.from_iterable(repeat(dists[i], times[i]) for i in range(len(times))))

    # Create a DataFrame for the current group
    aldistsdf = pd.DataFrame({
        'dists': all_dists_tnum,
        'Patient_ID': tn,
        'combo': combo
    })

    # Append the DataFrame to the list
    all_combos_dists_list.append(aldistsdf)

# Concatenate all DataFrames in the list once
all_combos_dists = pd.concat(all_combos_dists_list, ignore_index=True)


In [37]:
import pandas as pd
import numpy as np
# from scipy.stats import weibull_min

# def fit_weibull(data):
#     """
#     Fit a Weibull distribution to the data and return the shape and scale parameters.
#     """
#     params = weibull_min.fit(data, floc=0)  # Fix location parameter to zero for 2-parameter Weibull
#     shape = params[0]
#     scale = params[2]
#     return shape, scale

# Initialize a list to collect results
initial_params_list = []

for combo in all_distances_data['phenotype_combo'].unique():
    print(combo)
    combo_data = all_combos_dists[(all_combos_dists['combo'] == combo) & (all_combos_dists['dists'] < 100)]
    
    for tn in combo_data['Patient_ID'].unique():
        data = combo_data[combo_data['Patient_ID'] == tn]['dists']
        
        if len(data) > 0:
            shape, scale = fit_weibull(data)
            params = {
                'term': ['shape', 'scale'],
                'estimate': [shape, scale],
                'std.error': [np.nan, np.nan],  # Standard error is not calculated here
                'Patient_ID': [tn] * 2,
                'combo': [combo] * 2
            }
            initial_params_list.append(params)

# Convert the list of dictionaries to a DataFrame
initial_params = pd.DataFrame(
    {
        'term': [item for sublist in initial_params_list for item in sublist['term']],
        'estimate': [item for sublist in initial_params_list for item in sublist['estimate']],
        'std.error': [item for sublist in initial_params_list for item in sublist['std.error']],
        'Patient_ID': [item for sublist in initial_params_list for item in sublist['Patient_ID']],
        'combo': [item for sublist in initial_params_list for item in sublist['combo']]
    }
)

# Save the estimated parameters to a file
output_file = 'initial_params.csv'
initial_params.to_csv(output_file, sep=',', index=False)

# Display the resulting DataFrame
print(initial_params)


B cell_to_B cell
B cell_to_Cancer
B cell_to_Neutrophils
B cell_to_Tc
B cell_to_Th
B cell_to_Treg
Cancer_to_B cell
Cancer_to_Cancer
Cancer_to_Neutrophils
Cancer_to_Tc
Cancer_to_Th
Cancer_to_Treg
Neutrophils_to_B cell
Neutrophils_to_Cancer
Neutrophils_to_Neutrophils
Neutrophils_to_Tc
Neutrophils_to_Th
Neutrophils_to_Treg
Tc_to_B cell
Tc_to_Cancer
Tc_to_Neutrophils
Tc_to_Tc
Tc_to_Th
Tc_to_Treg
Th_to_B cell
Th_to_Cancer
Th_to_Neutrophils
Th_to_Tc
Th_to_Th
Th_to_Treg
Treg_to_B cell
Treg_to_Cancer
Treg_to_Neutrophils
Treg_to_Tc
Treg_to_Th
Treg_to_Treg
B cell_to_DCs cell
Cancer_to_DCs cell
DCs cell_to_B cell
DCs cell_to_Cancer
DCs cell_to_DCs cell
DCs cell_to_Neutrophils
DCs cell_to_Tc
DCs cell_to_Th
DCs cell_to_Treg
Neutrophils_to_DCs cell
Tc_to_DCs cell
Th_to_DCs cell
Treg_to_DCs cell
        term   estimate  std.error Patient_ID             combo
0      shape   1.299128        NaN  LUAD_D001  B cell_to_B cell
1      scale  23.916137        NaN  LUAD_D001  B cell_to_B cell
2      shape   1.

In [59]:
initial_params['combo'] =  initial_params['combo'].str.replace(' ','_')
initial_params

,term,estimate,std.error,Patient_ID,combo
0,shape,1.299128,NaN,LUAD_D001,B_cell_to_B_cell
1,scale,23.916137,NaN,LUAD_D001,B_cell_to_B_cell
2,shape,1.487152,NaN,LUAD_D002,B_cell_to_B_cell
3,scale,16.404950,NaN,LUAD_D002,B_cell_to_B_cell
4,shape,1.207969,NaN,LUAD_D003,B_cell_to_B_cell
...,...,...,...,...,...
32673,scale,77.287234,NaN,LUAD_D410,Treg_to_DCs_cell
32674,shape,6.446691,NaN,LUAD_D413,Treg_to_DCs_cell
32675,scale,46.330167,NaN,LUAD_D413,Treg_to_DCs_cell
32676,shape,4.352754,NaN,LUAD_D416,Treg_to_DCs_cell


In [60]:
# Save the estimated parameters to a file
initial_params.to_csv(f"{output_file}", sep=',', index=False)

In [23]:

# Old
# Recreate 1-NN histogram from coordinates (this is required for function "fitdistrplus")

all_combos_dists = pd.DataFrame()
for tn in all_distances_data['Patient_ID'].unique():
    for combo in all_distances_data['phenotype_combo'].unique():
        df_filtered = all_distances_data[(all_distances_data['Patient_ID'] == tn) & (all_distances_data['phenotype_combo'] == combo)]
        dists = [1] + df_filtered['distance_window'].tolist()
        times = [1] + df_filtered['new'].tolist()
        
        if len(times) == 2:
            dists = list(range(1, 299))
            times = [0] * 298
        
        all_dists_tnum = []
        for i in range(len(times)):
            all_dists_tnum.extend([dists[i]] * times[i])
        
        aldistsdf = pd.DataFrame({
            'dists': all_dists_tnum,
            'Patient_ID': [tn] * len(all_dists_tnum),
            'combo': [combo] * len(all_dists_tnum)
        })
        all_combos_dists = pd.concat([all_combos_dists, aldistsdf])

all_combos_dists

KeyboardInterrupt: 

In [36]:
# Fit a Weibull distribution by Maximum likelihood MLE [this is an initial estimation that will be optimized in step 3)]
initial_params = pd.DataFrame(columns=['term', 'estimate', 'std.error', 'Patient_ID', 'combo'])

for combo in all_distances_data['phenotype_combo'].unique():
    print(combo)
    for tn in all_combos_dists['Patient_ID'].unique():
        # if 'PanCK+' in combo or 'negative' in combo or 'Cancer' in combo or 'Negative' in combo:
        #     data = all_combos_dists[(all_combos_dists['combo'] == combo) & (all_combos_dists['Patient_ID'] == tn)]['dists']
        # else:
        data = all_combos_dists[(all_combos_dists['combo'] == combo) & (all_combos_dists['Patient_ID'] == tn) & (all_combos_dists['dists'] < 100)]['dists']
        
        if len(data) > 0:
            shape, scale = fit_weibull(data)
            params = pd.DataFrame({
                'term': ['shape', 'scale'],
                'estimate': [shape, scale],
                'std.error': [np.nan, np.nan],  # Standard error is not calculated here
                'Patient_ID': [tn] * 2,
                'combo': [combo] * 2
            })
            initial_params = pd.concat([initial_params, params])

# Save the estimated parameters to a file
initial_params.to_csv(output_file, sep=',', index=False)


B cell_to_B cell


C:\Users\jhroe\AppData\Local\Temp\ipykernel_13928\4004693982.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  initial_params = pd.concat([initial_params, params])


KeyboardInterrupt: 